# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


**========================================================================================================================**
Import packages which will be used in this notebook:

In [1]:
import os, sys, re
import pandas as pd
import numpy as np

Load the name mapping for the column names based on SeaBird's manual:

In [2]:
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [3]:
sbe_name_map.head()

,Short Name,Full Name,Friendly Name,Units,Notes/Comments
0,accM,Acceleration [m/s^2],acc M,m/s^2,NaN
1,accF,Acceleration [ft/s^2],acc F,ft/s^2,NaN
2,altM,Altimeter [m],alt M,m,NaN
3,altF,Altimeter [ft],alt F,ft,NaN
4,avgsvCM,"Average Sound Velocity [Chen-Millero, m/s]",avgsv-C M,"Chen-Millero, m/s",NaN


**========================================================================================================================**
Declare the directory paths to where the relevant information is stored:

In [399]:
basepath = '/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-09_AR-24_2017-10-22/'
leg = 'Leg 3 (AR24c)/'
water = 'Water Sampling/'
ctd = 'ctd/'

In [400]:
bottle_path = basepath+array+cruise+leg+ctd
water_path = basepath+array+cruise+water
salts_and_o2_path = water_path+'Pioneer-09_AR-24C_2017-10-22_Oxygen_Salinity_Sample_Data/'
sample_log_path = water_path+'Pioneer-09_AR-24C_CTD_Sampling_Log.xlsx'
nutrients_path = water_path+'Pioneer-09_AR-24C_2017-10-22_Nutrients_Sample_Data_2017-12-01_ver_1-00.xlsx'

In [401]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time [UTC]':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Filename':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude [degrees]':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude [degrees]':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise':cruise_id})
        else:
            pass
    
    return hdr

Get the path to the ctd-bottle data, load it, and parse it:

In [402]:
os.listdir(bottle_path)

['ar24c007.bl',
 'ar24c999.ros',
 'ar24c001.btl',
 'ar24c006.hdr',
 'ar24c007.hex',
 'ar24c007.hdr',
 'ar24c008.bl',
 'ar24c006.bl',
 'seasave_armstrong_2017june.psa',
 'ar24c002.hdr',
 'ar24c003.hex',
 'ar24c005.hex',
 'ar24c999.sum',
 'AR24C010.XMLCON',
 'AR24C008.XMLCON',
 'ar24c005.btl',
 'ar24c999.hex',
 'CTD_Summary.csv',
 'ar24c006.ros',
 'AR24C999.XMLCON',
 'ar24c001.hdr',
 'ar24c005.bl',
 'ar24c008.ros',
 'ar24c011.hdr',
 'ar24c002.btl',
 'ar24c012.bl',
 'AR24C011.XMLCON',
 'doc',
 'AR24C006.XMLCON',
 'ar24c005.ros',
 'ar24c008.hdr',
 'ar24c999.bl',
 'ar24c002.sum',
 'ar24c003.btl',
 'ar24c009.bl',
 'AR24C012.XMLCON',
 'ar24c005.hdr',
 'ar24c012.ros',
 'AR24C.xmlcon',
 'process',
 'ar24c999.hdr',
 'ar24c011.btl',
 'ar24c001.ros',
 'ar24c008.hex',
 'ar24c005.sum',
 'ar24c010.bl',
 'ar24c012.sum',
 'ar24c002.hex',
 'ar24c002.ros',
 'seasave_armstrong_2017oct.psa',
 'ar24c012.btl',
 'fixed_caution.dsa',
 'AR24C004.XMLCON',
 'ar24c999.btl',
 'ar24c008.btl',
 'AR24C007.XMLCON',
 'a

In [403]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(bottle_path) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(bottle_path+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
                
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
                
    #Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
    
    # Join the date and time for each measurement into a single item
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)

    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast'] = str(cast).zfill(3)
    
    # Add the header info back in
    for key in hdr.keys():
        df[key] = hdr[key]
        
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(bottle_path+outname)
    


In [404]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(bottle_path):
    if '.sum' in file:
        df = df.append(pd.read_csv(bottle_path+file))
    else:
        pass

In [405]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [406]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [407]:
df.sort_values(by=['Cast','Bottle Position'], inplace=True)
df.drop(columns='Unnamed: 0',inplace=True)
bottles = df

In [408]:
df.to_csv(bottle_path+'CTD_Summary.csv')

**========================================================================================================================**
### Process the Discrete Salinity and Oxygen Data
Next, I process the discrete salinity and oxygen sample data so that it is consistently named and ready to be merged with the existing data sets.

In [409]:
def clean_sal_files(dirpath):

    # Run check if files are held in excel format or csvs
    csv_flag = any(files.endswith('.SAL') for files in os.listdir(dirpath))
    if csv_flag:
        for filename in os.listdir(dirpath):
            sample = []
            salinity = []
            if filename.endswith('.SAL'):
                with open(dirpath+filename) as file:
                    data = file.readlines()
                    for ind1,line in enumerate(data):
                        if ind1 == 0:
                            strs = data[0].replace('"','').split(',')
                            cruisename = strs[0]
                            station = strs[1]
                            cast = strs[2]
                            case = strs[8]
                        elif int(line.split()[0]) == 0:
                            pass
                        else:
                            strs = line.split()
                            sample.append(strs[0])
                            salinity.append(strs[2])
                
                    # Generate a pandas dataframe to populate data
                    data_dict = {'Cruise':cruisename,'Station':station,'Cast':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
                    df = pd.DataFrame.from_dict(data_dict)
                    df.to_csv(file.name.replace('.','')+'.csv')
            else:
                pass
    
    else:
        # If the files are already in excel spreadsheets, they've been cleaned into a
        # logical tabular format
        pass
    

def process_sal_files(dirpath):
    
    # Check if the files are excel files or not
    excel_flag = any(files.endswith('SAL.xlsx') for files in os.listdir(dirpath))
    # Initialize a dataframe for processing the salinity files
    df = pd.DataFrame()
    if excel_flag:
        for file in os.listdir(dirpath):
            if 'SAL.xlsx' in file:
                df = df.append(pd.read_excel(dirpath+file))
        df.rename({'Sample':'Sample ID','Salinity':'Salinity [psu]','Niskin #':'Niskin','Case ID':'Case'}, 
                  axis='columns',inplace=True)
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    else:
        for file in os.listdir(dirpath):
            if 'SAL.csv' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
        df.drop(columns=[x for x in list(df.columns.values) if 'unnamed' in x.lower()],inplace=True)

    # Save the processed summary file for salinity
    df.to_csv(dirpath+'SAL_Summary.csv')
    
    
def process_oxy_files(dirpath):
    df = pd.DataFrame()
    for filename in os.listdir(dirpath):
        if 'oxy' in filename.lower() and filename.endswith('.xlsx'):
            df = df.append(pd.read_excel(dirpath+filename)) 
            # Rename and clean up the oxygen data to be uniform across data sets
    df.rename({'Niskin #':'Niskin','Sample#':'Sample ID','Oxy':'Oxygen [mL/L]','Unit':'Units'},
              axis='columns',inplace=True)
    df.dropna(inplace=True)
    df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
    df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
    df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    df['Cruise'] = df['Cruise'].apply(lambda x: x.replace('O','0'))
    
    # Save the processed summary file for oxygen
    df.to_csv(dirpath+'OXY_Summary.csv')

In [410]:
df

,Bottle Position,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],"Conductivity, 2 [S/m]",...,"Oxygen raw, SBE 43 [V]","Oxygen, SBE 43 [ml/l]","Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Filename,Start Latitude [degrees],Start Longitude [degrees],Start Time [UTC],Cast
0,1,Nov 06 2017 02:02:08,27.653,27.437,40.80186,-70.83050,15.7201,15.7223,4.056619,4.056655,...,2.2745,4.5284,5.70706,7.0380,17.2144 (avg),D:\Data\ar24c001.hex,40 48.11 N,070 49.83 W,2017-11-06T01:56:45Z,1
1,2,Nov 06 2017 02:04:07,2.551,2.531,40.80184,-70.83049,15.8029,15.8051,4.058227,4.058303,...,2.2673,4.4931,5.69927,7.7236,14.5103 (avg),D:\Data\ar24c001.hex,40 48.11 N,070 49.83 W,2017-11-06T01:56:45Z,1
0,1,Nov 06 2017 15:27:34,128.218,127.195,40.13676,-70.77464,14.0190,14.0184,4.276673,4.276125,...,1.6701,3.0531,5.78019,6.8938,17.8463 (avg),D:\Data\ar24c002.hex,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,2
1,2,Nov 06 2017 15:27:46,128.370,127.346,40.13676,-70.77466,14.0189,14.0147,4.276705,4.275772,...,1.6691,3.0503,5.78019,6.6892,18.7870 (avg),D:\Data\ar24c002.hex,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,2
2,3,Nov 06 2017 15:32:45,48.627,48.248,40.13674,-70.77466,18.7872,18.7836,4.675910,4.675233,...,2.3009,4.2899,5.29092,7.5072,15.3455 (avg),D:\Data\ar24c002.hex,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,2
3,4,Nov 06 2017 15:32:53,48.713,48.334,40.13674,-70.77465,18.7701,18.7781,4.674926,4.674983,...,2.2975,4.2779,5.29241,7.9674,13.6739 (avg),D:\Data\ar24c002.hex,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,2
4,5,Nov 06 2017 15:35:00,29.873,29.641,40.13676,-70.77466,19.1052,19.1069,4.718322,4.717934,...,2.3352,4.3367,5.25679,8.9142,10.7709 (avg),D:\Data\ar24c002.hex,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,2
5,6,Nov 06 2017 15:35:08,29.794,29.563,40.13676,-70.77466,19.1046,19.1060,4.718213,4.717841,...,2.3312,4.3299,5.25686,9.0222,10.4818 (avg),D:\Data\ar24c002.hex,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,2
6,7,Nov 06 2017 15:37:02,7.433,7.376,40.13674,-70.77466,18.3929,18.3857,4.607796,4.605725,...,2.3250,4.3682,5.33744,9.7873,8.6568 (avg),D:\Data\ar24c002.hex,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,2
7,8,Nov 06 2017 15:37:11,7.758,7.698,40.13674,-70.77466,18.2264,18.2249,4.583292,4.582228,...,2.3246,4.3743,5.35621,9.8610,8.4988 (avg),D:\Data\ar24c002.hex,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,2


In [411]:
os.listdir(salts_and_o2_path)

['012SAL.xlsx',
 '009SAL.csv',
 '004.SAL',
 '010.SAL',
 'SAL_Summary.csv',
 '011OXY.xlsx',
 '010SAL.csv',
 'OXY_Summary.csv',
 '004OXY.xlsx',
 '003SAL.csv',
 '011SAL.xlsx',
 '004SAL.csv',
 '009OXY.xlsx',
 '009SAL.xlsx',
 '004SAL.xlsx',
 '012OXY.xlsx',
 '002OXY.xlsx',
 '002SAL.xlsx',
 '002.SAL',
 '002SAL.csv',
 '010SAL.xlsx',
 '003SAL.xlsx',
 '003OXY.xlsx',
 '009.SAL',
 '010OXY.xlsx',
 '003.SAL']

In [412]:
# Now process the salts and oxygen data
    # Clean the salinity
clean_sal_files(salts_and_o2_path)
    # Process the salinity files
process_sal_files(salts_and_o2_path)
    # Process the oxygen files
process_oxy_files(salts_and_o2_path)

In [413]:
sal = pd.read_csv(salts_and_o2_path+'SAL_Summary.csv')
sal.drop(columns='Unnamed: 0', inplace=True)

In [414]:
sal

,Cruise,Station,Niskin,Case,Sample ID,Salinity [psu],Unit
0,AR24-C,12,1,R,9,34.5714,psu
1,AR24-C,12,2,R,10,34.5961,psu
2,AR24-C,12,3,R,11,34.7007,psu
3,AR24-C,12,4,R,12,34.6708,psu
4,AR24-C,12,5,R,13,34.4196,psu
5,AR24-C,12,6,R,14,34.4204,psu
6,AR24-C,12,7,R,15,34.4334,psu
7,AR24-C,12,8,R,16,34.4111,psu
8,AR24-C,11,1,R,1,35.1175,psu
9,AR24-C,11,2,R,2,35.1154,psu


In [415]:
oxy = pd.read_csv(salts_and_o2_path+'OXY_Summary.csv')
oxy.drop(columns='Unnamed: 0', inplace=True)

In [416]:
oxy

,Cruise,Station,Niskin,Case,Sample ID,Oxygen [mL/L],Units
0,AR24-C,11,1,B,1,4.126,mL/L
1,AR24-C,11,2,B,2,4.019,mL/L
2,AR24-C,11,3,B,3,3.091,mL/L
3,AR24-C,11,4,B,4,3.088,mL/L
4,AR24-C,11,5,B,5,5.262,mL/L
5,AR24-C,11,6,B,6,5.258,mL/L
6,AR24-C,11,7,B,7,5.253,mL/L
7,AR24-C,11,8,B,8,5.290,mL/L
8,AR24-C,4,1,T,1,3.737,mL/L
9,AR24-C,4,2,T,2,3.653,mL/L


**========================================================================================================================**
### CTD Sampling Log
Load in the CTD sampling log summary sheet. The summary sheet needs to be manually created and the data cleaned before attempting to import. Additionally, ensure that there is only one header line and that it is at the top of the file.

In [417]:
os.listdir(water_path)

['Pioneer-09_AR-24B_2017-10-22_Oxygen_Salinity_Sample_Data',
 'Pioneer-09_AR-24C_CTD_Sampling_Log.xlsx',
 'Pioneer-09_AR-24B_CTD_Sampling_Log.xlsx',
 'Pioneer-09_AR-24C_2017-10-22_Nutrients_Sample_Data_2017-12-01_ver_1-00.xlsx',
 'Pioneer-09_AR-24B_2017-10-22_Nutrients_Sample_Data_2017-12-01_ver_1-00.xlsx',
 'Pioneer-09_AR-24A_CTD_Sampling_Log.xlsx',
 'Pioneer-09_AR-24A_2017-10-22_Oxygen_Salinity_Sample_Data',
 'Pioneer-09_AR-24C_2017-10-22_Oxygen_Salinity_Sample_Data']

In [418]:
sample_log = pd.read_excel(sample_log_path,sheet_name='Summary',header=0)
sample_log.sort_values(by=['Station-Cast #','Niskin #'])

,Cruise ID,Station-Cast #,Target Asset,Start Latitude,Start Longitude,Start Date,Start Time,Bottom Depth [m],Date,Niskin #,...,Ph Bottle #,DIC/TA Bottle #,Salts Bottle #,Nitrate Bottle 1,Chlorophyll Brown Bottle #,Chlorophyll Filter Sample #,Unnamed: 19,Chlorophyll Brown Bottle Volume,Chlorophyll LN Tube,Comments
0,AR24-C,2,CNPM,40 08.204 N,70 46.477 W,2017-11-06,15:21:00,133,2017-11-06,1.0,...,1146.0,1147.0,A1,2-1,NaN,NaN,NaN,NaN,NaN,NaN
1,AR24-C,2,CNPM,40 08.204 N,70 46.477 W,2017-11-06,15:21:00,133,2017-11-06,2.0,...,NaN,NaN,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AR24-C,2,CNPM,40 08.204 N,70 46.477 W,2017-11-06,15:21:00,133,2017-11-06,3.0,...,NaN,1148.0,A3,2-2,1.0,NaN,NaN,NaN,NaN,NaN
3,AR24-C,2,CNPM,40 08.204 N,70 46.477 W,2017-11-06,15:21:00,133,2017-11-06,4.0,...,NaN,1149.0,A4,2-3,2.0,NaN,NaN,NaN,NaN,NaN
4,AR24-C,2,CNPM,40 08.204 N,70 46.477 W,2017-11-06,15:21:00,133,2017-11-06,5.0,...,NaN,1150.0,A5,2-4,3.0,NaN,NaN,NaN,NaN,chl max
5,AR24-C,2,CNPM,40 08.204 N,70 46.477 W,2017-11-06,15:21:00,133,2017-11-06,6.0,...,NaN,NaN,A6,NaN,4.0,NaN,NaN,NaN,NaN,NaN
6,AR24-C,2,CNPM,40 08.204 N,70 46.477 W,2017-11-06,15:21:00,133,2017-11-06,7.0,...,1151.0,1152.0,A7,2-5,5.0,NaN,NaN,NaN,NaN,NaN
7,AR24-C,2,CNPM,40 08.204 N,70 46.477 W,2017-11-06,15:21:00,133,2017-11-06,9.0,...,1153.0,NaN,A8,2-6,6.0,NaN,NaN,NaN,NaN,NaN
8,AR24-C,3,PMCO,40 05.769 N,70 53.009 W,2017-11-06,20:33:00,146,2017-11-06,1.0,...,1154.0,1155.0,A9,3-1,NaN,NaN,NaN,NaN,NaN,NaN
9,AR24-C,3,PMCO,40 05.769 N,70 53.009 W,2017-11-06,20:33:00,146,2017-11-06,2.0,...,NaN,1156.0,A10,3-2,NaN,NaN,NaN,NaN,NaN,NaN


In [419]:
def strip_x(x):
    if type(x) == str:
        x = x.replace('.','')
        return x
    else:
        return x

In [420]:
sample_log['Nitrate Bottle 1'] = sample_log['Nitrate Bottle 1'].apply(lambda x: strip_x(x))
sample_log['Start Date'] = sample_log['Start Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
sample_log['Start Time'] = sample_log['Start Time'].apply(lambda x: x.strftime('%H:%M:%S'))
sample_log['Start Time'] = sample_log['Start Date'] + 'T' + sample_log['Start Time'] + 'Z'

**========================================================================================================================**
### Merge the CTD-Bottle Data and Sample Log
The next step is to merge the CTD-Bottle data with the sample log using an outer merge based on the cast and niskin/bottle position. The outer merge means that all data will be retained, so that we do not accidentally discard either data-only casts or casts not recorded on the sample logs.

In [421]:
summary = bottles.merge(sample_log, how='outer', right_on=['Station-Cast #','Niskin #'], left_on=['Cast','Bottle Position'])
#summary = bottles.merge(sample_log, how='outer', right_on=['Station-Cast #'], left_on=['Cast'])

Fill in missing data based on the sample log info:

In [422]:
summary['Start Latitude [degrees]'] = summary['Start Latitude [degrees]'].fillna(value=summary['Start Latitude'])
summary['Start Longitude [degrees]'] = summary['Start Longitude [degrees]'].fillna(value=summary['Start Longitude'])
summary['Start Time [UTC]'] = summary['Start Time [UTC]'].fillna(value=summary['Start Date']+summary['Start Time'])
summary['Station-Cast #'] = summary['Station-Cast #'].fillna(value=summary['Cast'])
summary['Bottle Position'] = summary['Bottle Position'].fillna(value=summary['Niskin #']);

Eliminate the redundant columns:

In [423]:
summary.drop(columns=['Start Latitude','Start Longitude','Start Date','Start Time','Cast',
                      'Niskin #','Date','Time','Trip Depth'], inplace=True)

**========================================================================================================================**
Merge the discrete salinity and oxygen data into the sample_log based on the cast and niskin number. Do not use the sample bottle number - it is not stored in the processed discrete data we get back from the labs:

In [424]:
summary = summary.merge(sal, how='left', left_on=['Station-Cast #','Bottle Position'], right_on=['Station','Niskin'] )
summary['Salinity [psu]'] = summary['Salinity [psu]'].fillna(value=summary['Salts Bottle #'])
summary.rename(columns={'Salinity [psu]': 'Discrete Salinity [psu]'}, inplace=True)

Drop the unnecessary or extraneous columns:

In [425]:
summary.drop(columns=['Cruise','Station','Niskin','Case', 'Sample ID', 'Unit', 'Salts Bottle #'], inplace=True)

Oxygen data:

In [427]:
summary = summary.merge(oxy, how='left', left_on=['Station-Cast #','Bottle Position'], right_on=['Station','Niskin'] )
summary['Oxygen [mL/L]'] =  summary['Oxygen [mL/L]'].fillna(value=summary[' Oxygen Bottle #'])
summary.rename(columns={'Oxygen [mL/L]':'Discrete Oxygen [mL/L]'}, inplace=True)

In [428]:
summary.drop(columns=['Cruise','Station','Niskin','Case', 'Sample ID', 'Units', ' Oxygen Bottle #'], inplace=True)

**========================================================================================================================**
### Nutrients Data
Load the nutrients data (if it exists) and merge with the summary sheet. If the nutrients data has not been returned yet, we fill in the relevant columns with the data from the sampling logs.

In [430]:
try:
    nutrients = pd.read_excel(nutrients_path,header=0)
    nutrients
except IsADirectoryError:
    nutrients = pd.DataFrame(data=sample_log['Nitrate Bottle 1'])
    nutrients.rename(columns={'Nitrate Bottle 1':'Sample ID'}, inplace=True)
    columns = ['Sample ID','Cruise','Avg: Nitrate + Nitrite [µmol/L]','Avg: Ammonium [µmol/L]',
               'Avg: Phosphate [µmol/L]','Avg: Silicate [µmol/L]','Avg: Nitrite [µmol/L]','Avg: Nitrate [µmol/L]']
    for col in columns:
        if col not in nutrients.columns.values:
            nutrients[col] = nutrients['Sample ID']

In [431]:
nutrients.rename(columns=lambda x: x.replace('Avg:', 'Discrete'), inplace=True)
summary['Nitrate Bottle 1'] = summary['Nitrate Bottle 1'].apply(lambda x: str(x).replace(' ',''))

In [432]:
summary = summary.merge(nutrients, how='left', left_on='Nitrate Bottle 1', right_on='Sample ID')

In [433]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 0 to 73
Data columns (total 44 columns):
Bottle Position                              70 non-null float64
Date Time                                    69 non-null object
Pressure, Digiquartz [db]                    69 non-null float64
Depth [salt water, m]                        69 non-null float64
Latitude [deg]                               69 non-null float64
Longitude [deg]                              69 non-null float64
Temperature [ITS-90, deg C]                  69 non-null float64
Temperature, 2 [ITS-90, deg C]               69 non-null float64
Conductivity [S/m]                           69 non-null float64
Conductivity, 2 [S/m]                        69 non-null float64
Salinity, Practical [PSU]                    69 non-null float64
Salinity, Practical, 2 [PSU]                 69 non-null float64
Oxygen raw, SBE 43 [V]                       69 non-null float64
Oxygen, SBE 43 [ml/l]                        69 non-n

In [434]:
summary.drop(columns=['Sample ID','Cruise','Nitrate Bottle 1'], inplace=True)

**========================================================================================================================**
### Chlorophyll Data
If the Chlorophyll measurements have not been returned yet, we will generate a synthetic chlorophyll spreadsheet which substitutes the sample bottle numbers in place of the actual measurements. One complication is that the Chlorophyll sample # column title is not identical between cruises.

In [435]:
chl_path = water_path+''

In [436]:
try:
    chl = pd.read_excel(chl_path)
    chl.head()
except IsADirectoryError:
    # If there is no chlorophyll sheet yet, need to copy the bottle data into the final sample log
    chl = sample_log[['Station-Cast #','Chlorophyll Brown Bottle #','Chlorophyll Filter Sample #','Chlorophyll LN Tube']]
    chl.rename(columns={
        'Chlorophyll Brown Bottle #': 'Brown Bottle #',
        'Chlorophyll Filter Sample #': 'Discrete Chl (ug/l)',
        'Chlorophyll LN Tube':'Discrete Phaeo (ug/l)'
    }, inplace=True)

/home/andrew/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [437]:
chl.dropna(subset=['Brown Bottle #'], inplace=True)

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [438]:
summary = summary.merge(chl, how='left', left_on=['Station-Cast #','Chlorophyll Brown Bottle #'], right_on=['Station-Cast #','Brown Bottle #'])

In [439]:
summary.drop(columns=['Chlorophyll Brown Bottle #','Chlorophyll Filter Sample #','Chlorophyll LN Tube','Brown Bottle #',
                     'Chlorophyll Brown Bottle Volume'], inplace = True)

**========================================================================================================================**
### Carbon-System Measurements
If the Carbon system measurements have not been returned yet, we will generate a synthetic DIC spreadsheet which substitutes the sample bottle numbers in place of the actual measurements.

In [440]:
dic_path = water_path + ''

In [441]:
try:
    dic = pd.read_excel(dic_path,header=0)
    dic
except IsADirectoryError:
    dic = sample_log[['Station-Cast #','Niskin #','Ph Bottle #','DIC/TA Bottle #']]
    dic.rename(columns={
        'Station-Cast #':'CAST_NO',
        'Niskin #':'NISKIN_NO',
        'DIC/TA Bottle #':'DIC_UMOL_KG',
        'Ph Bottle #':'PH_TOT_MEA',
    }, inplace=True)
    columns = ['CAST_NO', 'NISKIN_NO','DIC_UMOL_KG', 'DIC_FLAG_W', 'TA_UMOL_KG',
       'TA_FLAG_W', 'PH_TOT_MEA', 'TMP_PH_DEG_C', 'PH_FLAG_W']
    for col in columns:
        if col not in dic.columns.values:
            if 'dic' in col.lower() or 'ta' in col.lower():
                dic[col] = dic['DIC_UMOL_KG']
            elif 'ph' in col.lower():
                dic[col] = dic['PH_TOT_MEA']
            else:
                dic[col] = np.nan

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [442]:
dic = dic[['CAST_NO', 'NISKIN_NO','DIC_UMOL_KG', 'DIC_FLAG_W', 'TA_UMOL_KG',
       'TA_FLAG_W', 'PH_TOT_MEA', 'TMP_PH_DEG_C', 'PH_FLAG_W']]
dic.rename(columns = {'DIC_UMOL_KG':'DIC [µmol/kg]',
               'DIC_FLAG_W':'DIC Flag',
               'TA_UMOL_KG':'Alkalinity [µmol/kg]',
               'TA_FLAG_W':'Alkalinity Flag',
               'PH_TOT_MEA':'pH [Total Scale]',
               'TMP_PH_DEG_C':'pH Analysis Temp [C]', 
              'PH_FLAG_W':'pH Flag'}, inplace=True)
# Add in the pCO2 columns, which we don't measure
dic['pCO2'] = np.nan
dic['pCO2 Flag'] = np.nan
dic['pCO2 Analysis Temp [C]'] = np.nan

dic.rename(columns=lambda x: 'Discrete ' + x, inplace=True)

In [443]:
summary = summary.merge(dic, how='left', left_on=['Station-Cast #','Bottle Position'], right_on=['Discrete CAST_NO','Discrete NISKIN_NO'])

In [444]:
summary.drop(columns=['Ph Bottle #','DIC/TA Bottle #','Discrete CAST_NO','Discrete NISKIN_NO'], inplace=True)

In [445]:
summary.rename(columns={'Date Time':'Bottle Closure'}, inplace=True)

In [446]:
summary.info();

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 0 to 73
Data columns (total 47 columns):
Bottle Position                              70 non-null float64
Bottle Closure                               69 non-null object
Pressure, Digiquartz [db]                    69 non-null float64
Depth [salt water, m]                        69 non-null float64
Latitude [deg]                               69 non-null float64
Longitude [deg]                              69 non-null float64
Temperature [ITS-90, deg C]                  69 non-null float64
Temperature, 2 [ITS-90, deg C]               69 non-null float64
Conductivity [S/m]                           69 non-null float64
Conductivity, 2 [S/m]                        69 non-null float64
Salinity, Practical [PSU]                    69 non-null float64
Salinity, Practical, 2 [PSU]                 69 non-null float64
Oxygen raw, SBE 43 [V]                       69 non-null float64
Oxygen, SBE 43 [ml/l]                        69 non-n

**========================================================================================================================**
Import the column order list and use fuzzy string matching to sort the data and save the data to an new Excel spreadsheet.

In [447]:
column_order = pd.read_excel(basepath+'column_order.xlsx')

In [448]:
column_order = tuple([x.replace('CTD','').strip() for x in column_order.columns.values])

In [449]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [450]:
results = {}
CTDsorted = pd.DataFrame()
for column in column_order:
    match = process.extractBests(column.replace('Discrete ','').replace('Calculated ',''),
                                 summary.columns.values, limit=2, score_cutoff=56, scorer=fuzz.ratio)
    if 'calculated' in column.lower():
        CTDsorted[column] = -9999999
    elif 'flag' in column.lower():
        if column not in ['Discrete DIC Flag','Discrete Alkalinity Flag','Discrete pCO2 Flag','Discrete pH Flag']:
            CTDsorted[column] = -9999999
        else:
            CTDsorted[column] = summary[column]
            results.update({column:match[0]})
    elif len(match) == 0:
        CTDsorted[column] = -9999999
    elif (match[0][0] not in [x[0] for x in results.values()]):
        CTDsorted[match[0][0]] = summary[match[0][0]]
        results.update({column:match[0]})
    elif len(match) == 1:
        CTDsorted[match[0][0]] = summary[match[0][0]]
        results.update({column:match[0]})
    else:
        CTDsorted[match[1][0]] = summary[match[1][0]]
        results.update({column:match[1]})
CTDsorted['Comments'] = summary['Comments']

In [451]:
cruise_id = list(set(CTDsorted['Cruise ID'].dropna()))
CTDsorted['Cruise ID'] = CTDsorted['Cruise ID'].fillna(value=cruise_id[0])

In [452]:
cruise_name = cruise.replace('/','')
current_date = pd.to_datetime(pd.datetime.now()).tz_localize(tz='US/Eastern').tz_convert(tz='UTC')
version = '1-01'

In [453]:
cruise_id

['AR24-C']

In [454]:
filename = '_'.join([cruise_name,cruise_id[0],'Discrete','Summary',current_date.strftime('%Y-%m-%d'),'ver',version,'.xlsx'])
filename

'Pioneer-09_AR-24_2017-10-22_AR24-C_Discrete_Summary_2019-06-26_ver_1-01_.xlsx'

In [455]:
CTDsorted.drop_duplicates(inplace=True)

In [456]:
CTDsorted

,Cruise ID,Station-Cast #,Target Asset,Start Latitude [degrees],Start Longitude [degrees],Start Time [UTC],Cast,Cast Flag,Bottom Depth [m],Filename,...,Calculated Alkalinity [µmol/kg],Calculated DIC [µmol/kg],Calculated pCO2 [µatm],Calculated pH,Calculated CO2aq [µmol/kg],Calculated bicarb [µmol/kg],Calculated CO3 [µmol/kg],Calculated Omega-C,Calculated Omega-A,Comments
0,AR24-C,1.0,NaN,40 48.11 N,070 49.83 W,2017-11-06T01:56:45Z,-9999999,-9999999,NaN,D:\Data\ar24c001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
1,AR24-C,1.0,NaN,40 48.11 N,070 49.83 W,2017-11-06T01:56:45Z,-9999999,-9999999,NaN,D:\Data\ar24c001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
2,AR24-C,2.0,CNPM,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,-9999999,-9999999,133.0,D:\Data\ar24c002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
3,AR24-C,2.0,CNPM,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,-9999999,-9999999,133.0,D:\Data\ar24c002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
4,AR24-C,2.0,CNPM,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,-9999999,-9999999,133.0,D:\Data\ar24c002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
5,AR24-C,2.0,CNPM,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,-9999999,-9999999,133.0,D:\Data\ar24c002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
6,AR24-C,2.0,CNPM,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,-9999999,-9999999,133.0,D:\Data\ar24c002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,chl max
7,AR24-C,2.0,CNPM,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,-9999999,-9999999,133.0,D:\Data\ar24c002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
8,AR24-C,2.0,CNPM,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,-9999999,-9999999,133.0,D:\Data\ar24c002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
9,AR24-C,2.0,NaN,40 08.21 N,070 46.48 W,2017-11-06T15:22:07Z,-9999999,-9999999,NaN,D:\Data\ar24c002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN


In [457]:
CTDsorted.fillna(value=-9999999,inplace=True)

In [458]:
CTDsorted.to_excel(basepath+array+cruise+filename)

In [303]:
os.listdir(basepath+array+cruise)

['Leg 1 (AR24a)',
 'Leg 3 (AR24c)',
 'Pioneer-09_AR24_Discrete_Summary_2019-06-21_ver_1-01_.xlsx',
 'Pioneer-09_Leg_3_AR24-C_Discrete_Summary_2019-06-14_ver_1-01_.xlsx',
 'Pioneer-09_Leg-3_AR24-C_Discrete_Summary_2019-03-13_ver_1-00_.xlsx',
 'Leg 2 (AR24b)',
 'AR-24A_discrete_sampling.xlsx',
 'Pioneer-09_AR-24_2017-10-22_AR24-C_Discrete_Summary_2019-06-26_ver_1-01_.xlsx',
 'Pioneer-09_Leg-2_AR24-B_Discrete_Summary_2019-03-13_ver_1-00_.xlsx',
 'Pioneer-09_AR-24_2017-10-22_AR24-B_Discrete_Summary_2019-06-26_ver_1-01_.xlsx',
 'Water Sampling',
 'Pioneer-09_Leg_2_AR24-B_Discrete_Summary_2019-06-14_ver_1-01_.xlsx',
 'Pioneer-09_AR-24_Discrete_Summary_2019-06-26_ver_1-01_.xlsx']

In [304]:
CTDsorted

,Cruise ID,Station-Cast #,Target Asset,Start Latitude [degrees],Start Longitude [degrees],Start Time [UTC],Cast,Cast Flag,Bottom Depth [m],Filename,...,Calculated Alkalinity [µmol/kg],Calculated DIC [µmol/kg],Calculated pCO2 [µatm],Calculated pH,Calculated CO2aq [µmol/kg],Calculated bicarb [µmol/kg],Calculated CO3 [µmol/kg],Calculated Omega-C,Calculated Omega-A,Comments
0,AR24-B,1,NaN,41 07.82 N,070 49.77 W,2017-10-27T23:53:29Z,-9999999,-9999999,NaN,D:\Data\AR24b001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
1,AR24-B,1,NaN,41 07.82 N,070 49.77 W,2017-10-27T23:53:29Z,-9999999,-9999999,NaN,D:\Data\AR24b001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
2,AR24-B,1,NaN,41 07.82 N,070 49.77 W,2017-10-27T23:53:29Z,-9999999,-9999999,NaN,D:\Data\AR24b001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
3,AR24-B,1,NaN,41 07.82 N,070 49.77 W,2017-10-27T23:53:29Z,-9999999,-9999999,NaN,D:\Data\AR24b001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
4,AR24-B,1,NaN,41 07.82 N,070 49.77 W,2017-10-27T23:53:29Z,-9999999,-9999999,NaN,D:\Data\AR24b001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
5,AR24-B,1,NaN,41 07.82 N,070 49.77 W,2017-10-27T23:53:29Z,-9999999,-9999999,NaN,D:\Data\AR24b001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
6,AR24-B,2,NaN,40 48.01 N,070 49.74 W,2017-10-28T02:13:06Z,-9999999,-9999999,NaN,D:\Data\AR24b002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
7,AR24-B,2,NaN,40 48.01 N,070 49.74 W,2017-10-28T02:13:06Z,-9999999,-9999999,NaN,D:\Data\AR24b002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
8,AR24-B,2,NaN,40 48.01 N,070 49.74 W,2017-10-28T02:13:06Z,-9999999,-9999999,NaN,D:\Data\AR24b002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
9,AR24-B,2,NaN,40 48.01 N,070 49.74 W,2017-10-28T02:13:06Z,-9999999,-9999999,NaN,D:\Data\AR24b002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN


,Sample ID,Cruise,Discrete Nitrate+Nitrite [µmol/L],Discrete Ammonium [µmol/L],Discrete Phosphate [µmol/L],Discrete Silicate [µmol/L],Discrete Nitrite [µmol/L],Discrete Nitrate [µmol/L]
0,3-1,AR24B,13.3523,0.257,0.814416,8.26803,<0.04,13.3123
1,3-2,AR24B,15.7272,0.2775,0.969869,9.69607,<0.04,15.6872
2,3-3,AR24B,20.1773,0.267,1.12337,10.6867,<0.04,20.1373
3,3-4,AR24B,<0.04,0.33,<0.009,<0.03,<0.04,<0.04
4,3-5,AR24B,<0.04,0.2115,<0.009,<0.03,<0.04,<0.04
5,3-6,AR24B,<0.04,0.363,<0.009,<0.03,<0.04,<0.04
6,7-1,AR24B,8.77223,0.392,0.483957,4.66685,<0.04,8.73223
7,7-2,AR24B,10.2713,0.467,0.57586,5.45055,<0.04,10.2313
8,7-3,AR24B,4.92895,0.3105,0.298685,1.78612,<0.04,4.88895
9,7-4,AR24B,<0.04,0.369,<0.009,<0.03,<0.04,<0.04
